In [32]:
from glob import glob
import pandas as pd
import numpy as np
import re

In [132]:
from plotly import express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
# 캐글 api 파일 저장
import os
home_folder = os.getenv('HOME')
kaggle_config_folder = os.path.join(home_folder, '.kaggle')
kaggle_api_file_path = os.path.join(kaggle_config_folder, 'kaggle.json')
os.makedirs(kaggle_config_folder, exist_ok=True)
with open(kaggle_api_file_path, 'w') as f:
    f.write('{"username":"dantekwak","key":"3ecadc3b1192d996e8254cecad78e697"}')

* https://archive.ics.uci.edu/dataset/502/online+retail+ii

In [11]:
from kaggle.api.kaggle_api_extended import KaggleApi

dataset = 'sanlian/online-retail-dataset'
path = 'datasets/online-retail/'

api = KaggleApi()
api.authenticate()
api.dataset_download_files(dataset, path=path, unzip=True)

In [52]:
df = pd.read_excel(glob(f'{path}/*.xlsx')[0])
df.head()



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


| | 컬럼 | 설명 |
| --- | --- | --- |
| 0 | Invoice | 송장 번호 |
| 1 | StockCode | 재고 코드 |
| 2 | Description | 상품 설명 |
| 3 | Quantity | 수량 |
| 4 | InvoiceDate | 송장 날짜 |
| 5 | Price | 가격 |
| 6 | Customer ID | 고객 ID |
| 7 | Country | 국가 |


### 데이터 초기 전처리

In [53]:
# 대문자의 경우 _와 소문자를 붙인 문자로 치환
df.columns = ['invoice',
 'stock_code',
 'description',
 'quantity',
 'invoice_date',
 'price',
 'cust_id',
 'country']


#### 결측 제거

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       525461 non-null  object        
 1   stock_code    525461 non-null  object        
 2   description   522533 non-null  object        
 3   quantity      525461 non-null  int64         
 4   invoice_date  525461 non-null  datetime64[ns]
 5   price         525461 non-null  float64       
 6   cust_id       417534 non-null  float64       
 7   country       525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


* 사용자 ID 가 누락된 데이터를 제거

In [148]:
# 취소, 할인, 환불 등의 마이너스 매출 데이터 확인
(df.quantity < 0).value_counts()

quantity
False    513135
True      12326
Name: count, dtype: int64

* 테스트 데이터 제거

In [169]:
# cust_id가 누락된 데이터 제거 (사유: 실제 사례에서는 해당 데이터들은 회계 차대변을 처리하거나, 재고마감을 위해 전산 처리하는 경우들이 있음)
transaction_df = df[df.cust_id.isnull() == False]

In [168]:
transaction_df = transaction_df[transaction_df.stock_code.apply(lambda x : (x.find('TEST') == -1 ) if isinstance(x, str) else True)]

In [170]:
# cust_id는 문자열로 정리
transaction_df['cust_id'] = transaction_df['cust_id'].astype(int).astype(str)

In [173]:
# 단위가격 x 수량 = 총 가격
transaction_df['total_price'] = transaction_df['quantity'] * transaction_df['price']

### RFM 분석

* 실무에서 RFM 분석을 통해서 알고자 하는것. 
    - VIP를 관리하기 위해
    - 높은 충성도 고객군을 알기 위해
    - 이탈이 예상되는 고객군을 탐지하기 위해
    - 이미 이탈된 고객한 알기 위해.
    - 어떤 고객군이 마케팅 에너지를 쓸필요가 있는지 알기 위해
    - 준비중인 프로모션에 반응이 예상되는 고객 세그먼트를 알기 위해

In [271]:
rfm_df = pd.DataFrame()

#### 최신성(Recency)

In [272]:
transaction_df.head()

,invoice,stock_code,description,quantity,invoice_date,price,cust_id,country,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [273]:
# 최신 주문일수록 높은 정수가 저장되도록
min_date = transaction_df['invoice_date'].min()
max_date = transaction_df['invoice_date'].max()
agg_df = transaction_df.groupby('cust_id').agg(
    recency_date = ('invoice_date', lambda x : max(x)),
    recency_by_days=('invoice_date', lambda x : (max(x) - min_date).days),
    # 최근 7일 이내 구매
    # recency_by_week=('invoice_date', lambda x : (max_date - max(x)).days < 7),
    # 최근 30일 이내 구매
    # recency_by_month=('invoice_date', lambda x : (max_date - max(x)).days < 30)
)
rfm_df = pd.concat([rfm_df, agg_df], axis=1)


In [274]:
rfm_df.head()

,recency_date,recency_by_days
cust_id,,
12346,2010-10-04 16:33:00,307
12347,2010-12-07 14:57:00,371
12348,2010-09-27 14:59:00,300
12349,2010-10-28 08:23:00,331
12351,2010-11-29 15:23:00,363


#### 빈도(Frequency)

In [275]:
# 고객별 방문 빈도 계산
cust_visit_counts = transaction_df[['cust_id', 'invoice']].drop_duplicates()['cust_id'].value_counts()

# 1x2 서브플롯 생성
fig = make_subplots(rows=1, cols=2, subplot_titles=("히스토그램", "박스플롯"))

# 히스토그램 추가
fig.add_trace(
    go.Histogram(x=cust_visit_counts, name="히스토그램"),
    row=1, col=1
)

# 박스플롯 추가
fig.add_trace(
    go.Box(y=cust_visit_counts, name="박스플롯"),
    row=1, col=2
)

# 레이아웃 업데이트
fig.update_layout(title_text="사용자 방문 빈도 분포", width=1200, showlegend=False)
fig.show()


In [276]:
# 취소분은 구매횟수에 반영하지 않는다.
agg_df = transaction_df[transaction_df.apply(lambda df : df['total_price'] > 0, axis=1)] \
                [['cust_id', 'invoice', 'invoice_date']].drop_duplicates().groupby('cust_id').agg(
    # 전체 구매횟수
    frequency=('invoice', 'count'),
    # 최근 7일간 구매횟수
    frequency_by_week=('invoice_date', lambda x : sum((max_date - x).apply(lambda x : x.days < 7))),
    # 최근 30일간 구매횟수
    frequency_by_month=('invoice_date', lambda x : sum((max_date - x).apply(lambda x : x.days < 30)))
)
rfm_df = rfm_df.join(agg_df, how='left')
rfm_df.head()


,recency_date,recency_by_days,frequency,frequency_by_week,frequency_by_month
cust_id,,,,,
12346,2010-10-04 16:33:00,307,11.0,0.0,0.0
12347,2010-12-07 14:57:00,371,2.0,1.0,1.0
12348,2010-09-27 14:59:00,300,1.0,0.0,0.0
12349,2010-10-28 08:23:00,331,3.0,0.0,0.0
12351,2010-11-29 15:23:00,363,1.0,0.0,1.0


#### 구매크기(Monetary)

In [280]:
# 1x2 서브플롯 생성
fig = make_subplots(rows=1, cols=2, subplot_titles=("고객별 총 구매 금액 분포", "고객별 총 구매상품수량 분포"))

# 고객별 총 구매 금액 분포 추가
cust_total_price = transaction_df.groupby('cust_id')['total_price'].sum()
fig.add_trace(
    go.Histogram(x=cust_total_price, name="총 구매 금액"),
    row=1, col=1
)

# cust_total_price.describe() 을 annotation으로 텍스트 그리기
desc1 = cust_total_price.describe()
for key, value in desc1.items():
    fig.add_annotation(text=f'{key}: {value:.2f}', align='left', showarrow=False, xref='paper', yref='paper', x=0.1, y=0.95 - 0.12 * list(desc1.keys()).index(key), bordercolor='black', borderwidth=1)
# 고객별 총 구매상품수량 분포 추가
cust_total_quantity = transaction_df.groupby('cust_id').agg(total_quantity=('quantity', 'sum'))
fig.add_trace(
    go.Histogram(x=cust_total_quantity['total_quantity'], name="총 구매상품수량"),
    row=1, col=2
)
desc2 = cust_total_quantity['total_quantity'].describe()
for key, value in desc2.items():
    fig.add_annotation(text=f'{key}: {value:.2f}', align='left', showarrow=False, xref='paper', yref='paper', x=0.7, y=0.95 - 0.12 * list(desc2.keys()).index(key), bordercolor='black', borderwidth=1)


# 레이아웃 업데이트
fig.update_layout(height=500, width=1200, showlegend=False)
fig.show()

In [281]:
# 객단가 분석 : 고객별 트랜잭션(invoice) 총합계의 평균
avg_price_per_customer = transaction_df.groupby(['cust_id', 'invoice'])

.agg(total_price=('total_price', 'sum')).groupby('cust_id')

.agg(cust_unit_price=('total_price', 'mean'))

# 분포 그리기
fig = px.histogram(avg_price_per_customer, x='cust_unit_price', title='고객별 평균 객단가 분포')

# describe 정보
desc = avg_price_per_customer.describe()
for index, (key, value) in enumerate(desc.iterrows()) :
    # value를 문자열로 변환하여 포맷 문제 해결
    fig.add_annotation(text=f'{key}: {value.values[0]}', align='left', showarrow=False, xref='paper', yref='paper', x=0.05, y=0.95 - 0.12 * index, bordercolor='black', borderwidth=1)
fig.show()


In [282]:
rfm_df = rfm_df.join(cust_total_price, how='left')
rfm_df = rfm_df.join(cust_total_quantity, how='left')
rfm_df = rfm_df.join(avg_price_per_customer, how='left')

In [286]:
rfm_df.drop(columns=['recency_date'], inplace=True)

In [287]:
rfm_df.head()

,recency_by_days,frequency,frequency_by_week,frequency_by_month,total_price,total_quantity,cust_unit_price
cust_id,,,,,,,
12346,307,11.0,0.0,0.0,-64.68,52,-4.3120
12347,371,2.0,1.0,1.0,1323.32,828,661.6600
12348,300,1.0,0.0,0.0,222.16,373,222.1600
12349,331,3.0,0.0,0.0,2646.99,988,661.7475
12351,363,1.0,0.0,1.0,300.93,261,300.9300


In [ ]:
rfm_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4383 entries, 12346 to 18287
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   recency_by_days     4383 non-null   int64  
 1   frequency           4312 non-null   float64
 2   frequency_by_week   4312 non-null   float64
 3   frequency_by_month  4312 non-null   float64
 4   total_price         4383 non-null   float64
 5   total_quantity      4383 non-null   int64  
 6   cust_unit_price     4383 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 403.0+ KB


### #과제2. RFM 고객 세그먼트
- 아래 표는 RFM으로 고객군을 나누는 예시를 보여줍니다.
- 본인이 생각하는 세그먼트 전략에 따라 나눠주세요. (아래 분류대로 되지 않아도되며 최소 4개의 세그먼트를 나눠주세요.)
- 각 고객군 rfm_df 데이터를 탐색하여 냐눠서 rfm_df['segmentation] 열에 저장해주세요.
- 본인의 세그먼트 전략의 근거를 간략히 서술해주세요. (200자 내외)
- 제출자료 : rfm_df 를 csv파일로 제출. 주피터노트북은 html로 변환하여 함께 제출.
- 제출방법 : e-campus 제4주차 과제란 에 업로드
- 공지 : 본 과제는 지난 시간 과제#1과 함께 중간고사 평가에 반영됩니다.

| 고객 세그먼트   | R (최근성)                            | F (빈도)                               | M (금전적 가치)                         | 설명                                                                                                   |
|------------------|------------------------------------|------------------------------------|------------------------------------|-----------------------------------------------------------------------------------------------------|
| VIP            | 매우 높음                             | 매우 높음                             | 매우 높음                             | 최근에 구매를 완료하였으며, 구매 빈도 및 지출 금액 모두에서 최상위권. 브랜드에 대한 강력한 충성도. 신제품의 초기 채택자. 브랜드 홍보대사 |
| 충성 고객      | 높음                                | 높음                                | 높음                                | 정기적으로 사이트에서 구매를 진행하며, 상당한 금액을 지출. 제안에 대한 반응성이 높아 상향 판매 전략이 효과적                      |
| 성장 고객    | 높음                                | 중간                                | 중간                                | 최근에 사이트에서 구매를 완료한 경험. 구매 빈도 및 지출 금액에서 잠재가능성 보임. 로열티 프로그램 참여를 유도하여 더 높은 수준의 충성도 구축 가능성           |
| 신규 고객          | 매우 높음                             | 낮음                                | 낮음                                | 최근 구매 경험은 있으나, 빈도수와 지출 금액이 낮아 앞으로의 충성도 구축에 초점을 맞출 필요                                     |
| 유망 고객            | 중간                                | 낮음                                | 낮음                                | 최근 구매 경험이 있으나, 지출 금액이 상대적으로 낮아 브랜드 인지도를 높이고 추가 구매를 유도하는 전략이 필요.                               |
| 주의 고객       | 중간                                | 중간                                | 중간                                | 최신성, 빈도, 금전적 가치 모두 평균적인 수준. 최근 구매 경험이 없어 재활성화 전략이 요구됨.                                             |
| 매우주의 고객       | 낮음                                | 높음                                | 높음                                | 과거, 대규모 및 빈번한 구매 경험이 있었으나 최근 활동이 없어, 다시 브랜드와의 연결을 회복시킬 전략이 필요.                                |
| 휴면 고객       | 낮음                                | 낮음                                | 낮음                                | 최근성, 빈도, 금전적 가치 모두 낮아, 재활성화하지 않으면 이탈 위험이 높은 고객군.                                                    |
| 위험 고객              | 낮음                                | 낮음                                | 높음                                | 과거에 크고 빈번한 구매가 있었으나, 최근에는 활동이 없는 고객군으로, 개인화 전략으로 다시 브랜드로 유도할 필요.                           |
| 이탈 고객            | 낮음                                | 매우 낮음                             | 매우 낮음                             | 마지막 구매가 매우 오래전. 구매 빈도 및 지출 금액도 매우 낮아 브랜드와의 다시 연결을 하기위한 전략 필요.                            |
| 블랙 고객          | 매우 낮음                             | 매우 낮음                             | 매우 낮음                             | 최신성, 빈도 및 금전적 가치 모두에서 최하위 수준에 위치한 고객군. 마케팅 성공 확률이 낮음.          |


In [293]:
rfm_df.fillna(0, inplace=True)

### 클러스터링


| 클러스터링 유형 | 알고리즘 | 설명| 장점 | 단점 |
|---|---|---|---|---|
| **밀도 기반 클러스터링** | DBSCAN, OPTICS | 데이터 포인트의 밀집도를 기반으로 클러스터를 형성합니다. | - 불규칙한 형태의 클러스터도 찾을 수 있음<br>- 이상치 감지 가능 | - 매개변수 설정이 중요하며 복잡할 수 있음<br>- 속도가 느림 |
| **분포 기반 클러스터링** | 가우스 혼합 모델(GMM) | 데이터가 특정 분포를 따른다고 가정하고 클러스터를 형성합니다. | - 분포 가정이 맞을 경우 정확한 클러스터링 가능 | - 분포 가정이 맞지 않을 경우 성능 저하 |
| **중심 기반 클러스터링** | K-평균, K-중앙값 | 클러스터 중심을 정의하고 각 데이터 포인트를 가장 가까운 중심에 할당합니다. | - 계산이 빠르고 간단함<br>- 널리 사용됨 | - 클러스터 수를 미리 정해야 함<br>- 클러스터 형태가 구형일 때 최적 |
| **계층 기반 클러스터링** | 응집적 계층 클러스터링, BIRCH | 데이터 포인트를 개별 클러스터로 보고 점차적으로 병합하거나 나누어 전체 구조를 계층적으로 조직합니다. | - 클러스터 수를 사전에 정할 필요 없음<br>- 계층적 구조 명확히 파악 가능 | - 대규모 데이터셋에 대해서는 계산량이 많음 |




#### DBSCAN 
- 특징
    - DBSCAN은 데이터의 밀도를 기반으로 클러스터를 형성하는 방법
    - 아웃라이어가 많은 데이터셋의 경우 K-Means보다 품질이 우수

- 주요 파라미터
    - R (Radius of Neighborhood)
    - M (Min number of Neighbors)
    - Core Point : 주변 영역(r) 내에 최소 데이터 수(minPts) 이상을 포함하는 데이터 포인트.
    - 경계점(Border Point) : 한 클러스터에 속하지만 핵심 객체는 아닌 데이터 포인트.
    - 노이즈포인트(Noise Point) : 어떤 클러스터에도 속하지 않는 데이터 포인트로, 어떤 핵심 객체의 밀도에도 도달할 수 없습니다. (DBSCAN은 아웃라이어 탐지에도 사용됨.)
- 수행전략
    - 데이터 포인트별 R 범위내에 Border Point 체크
    - M개 이웃이 R내에 존재한다면, Core Point로 지정하고 R내의 나머지는 Border Point로 지정
    - 어디에도 속하지 못한 Noise Point는 아웃라이어로 처리
    - Core Point 간의 반경이 R 내에 속할경우, 클러스터 병합



![DBSCAN 알고리즘 예시](https://pic2.zhimg.com/v2-58145667049e230f95e07c3dfbfd31ad_b.gif)


In [303]:
from sklearn.datasets import make_classification
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import metrics


In [304]:
columns = rfm_df.columns

# 데이터 전처리
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_df[columns])

# DBSCAN 모델 생성 및 학습
# eps : 두 데이터 포인트가 이웃으로 간주되기 위한 최대 거리
# min_samples : 핵심 포인트가 되기 위해 필요한 이웃의 최소 수
dbscan = DBSCAN(eps=0.3, min_samples=10)
clusters = dbscan.fit_predict(rfm_scaled)

# 클러스터 결과 추가
rfm_df['dbscan_clusters'] = clusters

rfm_df.head()

,recency_by_days,frequency,frequency_by_week,frequency_by_month,total_price,total_quantity,cust_unit_price,dbscan_clusters
cust_id,,,,,,,,
12346,307,11.0,0.0,0.0,-64.68,52,-4.3120,-1
12347,371,2.0,1.0,1.0,1323.32,828,661.6600,-1
12348,300,1.0,0.0,0.0,222.16,373,222.1600,0
12349,331,3.0,0.0,0.0,2646.99,988,661.7475,0
12351,363,1.0,0.0,1.0,300.93,261,300.9300,1


In [314]:
# 모델 평가
print('추정된 클러스터 수: %d' % np.unique(clusters).size)
print("Silhouette Score: %0.3f" % metrics.silhouette_score(rfm_scaled, clusters))


추정된 클러스터 수: 8
Silhouette Score: 0.288


* 실루엣 스코어 알고리즘 설명 참고 블로그
    : https://losskatsu.github.io/machine-learning/silhouette-score/#1-%EC%84%9C%EB%A1%A0

In [319]:
# tsne를 plotly로 3d로 시각화
from sklearn.manifold import TSNE

# t-SNE를 사용하여 데이터를 3차원으로 축소
tsne = TSNE(n_components=3, random_state=42)
tsne_result = tsne.fit_transform(rfm_scaled)

# 3D 시각화
fig = px.scatter_3d(x=tsne_result[:,0], y=tsne_result[:,1], z=tsne_result[:,2], color=clusters, size_max=2)
fig.update_layout(title='t-SNE 3D 시각화', scene=dict(xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Z')), width=800, height=800)
fig.show()


#### K-Means
- 특징
    - K-Means는 주어진 데이터를 K개의 클러스터로 묶는 알고리즘으로, 각 클러스터의 중심(centroid)을 기준으로 데이터 포인트 할당.
    - 빠른 수행 속도와 일반적인 클러스터링 문제에 쉽게 적용할 수 있다는 장점이 있습니다. 하지만, 클러스터의 수(K)를 미리 지정해야 하는 단점 존재.
    
- 주요 파라미터
    - K (클러스터의 수): 데이터를 나눌 클러스터의 개수 의미
    - Centroid: 각 클러스터의 중심점을 의미하며, 클러스터에 속하는 데이터 포인트들의 평균 위치에 해당함.
    - 클러스터 할당: 각 데이터 포인트를 가장 가까운 센트로이드를 기준으로 클러스터에 할당.
    - 센트로이드 업데이트: 클러스터에 할당된 데이터 포인트들의 평균 위치로 센트로이드를 업데이트.
    
- 수행전략
    - 초기 센트로이드(K개)를 무작위로 선정.
    - 각 데이터 포인트를 가장 가까운 센트로이드에 할당하여 클러스터를 형성.
    - 할당된 클러스터를 기반으로 새로운 센트로이드 계산.
    - 센트로이드의 변화가 없을 때까지 2-3단계 반복.
    - 클러스터의 수(K)를 변경하면서 최적의 클러스터링 결과를 도출.

In [320]:
from sklearn.cluster import KMeans 

##### 1) 클러스터수 결정방법1: 엘보우 메소드
- 클러스터 내 분산의 정의
    - 클러스터 내 분산(WCSS, Within-Cluster Sum of Squares)은 클러스터 내의 모든 데이터 포인트와 클러스터 중심점 사이의 거리의 제곱합으로 계산됨. 
    - 이 값은 클러스터가 얼마나 밀집되어 있는지를 나타냄.
- 클러스터 수 K에 따른 변화
    - 클러스터 수 K를 1부터 증가시키면서 각각에 대한 WCSS 값을 계산됨.
    -  일반적으로 K가 증가함에 따라 WCSS는 감소.
- "엘보우" 지점 탐색
    - WCSS의 감소율이 급격히 떨어지는 지점 탐색.
    - 이 지점은 K를 증가시킴으로써 얻을 수 있는 이득(분산 감소)이 상대적으로 적어지는 지점. 
    - 즉, 이 지점 이후로는 클러스터 수를 늘려도 클러스터의 밀집도가 크게 개선되지 않는다는 것을 의미.
- 최적의 K결정
    - 엘보우 지점에서의 K값을 최적의 클러스터 수로 결정. 
    - 클러스터링의 효율성과 효과 사이의 균형을 고려한 선택임.

In [355]:
# rfm_scaled을 KMeans를 사용하여 클러스터링
inertia_values = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(rfm_scaled)
    inertia_values.append(kmeans.inertia_)

# 엘보우 방법을 사용하여 최적의 클러스터 수 찾기
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, 11)), y=inertia_values, mode='lines+markers', name='Inertia'))
# x축 3번째 값(Number of clusters)에 빨간색 수직선 추가
fig.add_shape(type="line", x0=3, y0=0, x1=3, y1=max(inertia_values), line=dict(color="Red", width=2))
# 엘보우 지점이라고 annotation 
fig.add_annotation(x=3, y=max(inertia_values) * 1.05, text="Elbow Point", showarrow=False, xref='x', yref='y',
                  textangle=0, xanchor='center', yanchor='middle', font=dict(color='red', size=14))
fig.update_layout(title='엘보우 메소드', xaxis_title='클러스터 수', yaxis_title='Inertia')
fig.show()



##### 2) 클러스터수 결정방법2: 실루엣 스코어링
- 클러스터 내의 데이터 포인트가 얼마나 잘 모여 있는지와 다른 클러스터와는 얼마나 멀리 떨어져 있는지를 동시에 고려
- 실루엣 점수의 범위는 -1에서 1 사이이며, 1에 가까울수록 클러스터링이 잘 되었다고 평가

In [343]:
silhouette_scores = []
for k in range(2, 11): # 클러스터 수 2부터 10까지 시도
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled)
    cluster_labels = kmeans.labels_
    silhouette_avg = metrics.silhouette_score(rfm_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)

In [345]:
optimal_k = np.argmax(silhouette_scores) + 2

In [352]:
# 실루엣 스코어를 plotly로 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(2, 11)), y=silhouette_scores, mode='lines+markers', name='Silhouette Score'))
# 최적의 k 지점에 빨간색 수직선 추가
fig.add_shape(type="line", x0=optimal_k, y0=0.3, x1=optimal_k, y1=max(silhouette_scores)* 1.02, line=dict(color="Red", width=2))
# 최적의 k 지점에 annotation 추가
fig.add_annotation(x=optimal_k, y=max(silhouette_scores) * 1.05, text="Optimal k", showarrow=False, xref='x', yref='y',
                  textangle=0, xanchor='center', yanchor='middle', font=dict(color='red', size=14))

fig.update_layout(title='클러스터 수에 따른 실루엣 점수', xaxis_title='클러스터 수', yaxis_title='실루엣 점수')
fig.show()


본 데이터셋에서는 클러스터 수를 균형을 잡아 6으로 선정함

In [357]:
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(rfm_scaled)
rfm_df['kmeans_clusters'] = kmeans.labels_
rfm_df.head()


,recency_by_days,frequency,frequency_by_week,frequency_by_month,total_price,total_quantity,cust_unit_price,dbscan_clusters,kmeans_clusters
cust_id,,,,,,,,,
12346,307,11.0,0.0,0.0,-64.68,52,-4.3120,-1,0
12347,371,2.0,1.0,1.0,1323.32,828,661.6600,-1,0
12348,300,1.0,0.0,0.0,222.16,373,222.1600,0,0
12349,331,3.0,0.0,0.0,2646.99,988,661.7475,0,0
12351,363,1.0,0.0,1.0,300.93,261,300.9300,1,0


In [360]:
# t-SNE를 사용하여 데이터를 3차원으로 축소
tsne = TSNE(n_components=3, random_state=42)
tsne_result = tsne.fit_transform(rfm_scaled)

# 3D 시각화
fig = px.scatter_3d(x=tsne_result[:,0], y=tsne_result[:,1], z=tsne_result[:,2], color=rfm_df['kmeans_clusters'], size_max=2)
fig.update_layout(title='t-SNE 3D 시각화', scene=dict(xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Z')), width=800, height=800)
fig.show()


#### 가우시안 혼합 모델 (Gaussian Mixture Model, GMM)
- 특징
    - 가우시안 혼합 모델(GMM)은 데이터가 여러 개의 가우시안 분포(정규 분포)의 혼합으로 구성되어 있다고 가정하는 확률적 클러스터링 알고리즘.
    - 각 클러스터는 다변수 가우시안 분포를 따르며, 이를 통해 클러스터링을 수행합니다. GMM은 데이터 내에 숨겨진 가우시안 분포의 파라미터(평균, 공분산)를 추정.
    - K-Means에 비해 더 유연한 클러스터 형태를 모델링할 수 있으며, 클러스터의 크기, 모양, 밀도가 서로 다를 때 유용. 하지만, 계산 복잡도가 높고 초기값에 민감함.
    - GMM은 각 데이터 포인트가 여러 클러스터에 속할 확률을 제공함으로써, 클러스터링을 더욱 세밀하게 수행할 수 있는 장점

- 주요 파라미터
    - 클러스터의 수: 혼합 모델에 사용할 가우시안 분포의 개수
    - 평균(μ): 각 가우시안 분포의 중심값
    - 공분산(Σ): 각 가우시안 분포의 형태와 방향을 결정. 클러스터의 모양과 크기 제어
    - 혼합 계수(π): 각 가우시안 분포가 전체 데이터 집합에서 차지하는 비율

- 수행전략
    - 초기 파라미터(평균, 공분산, 혼합 계수)를 추정
    - E 단계(Expectation): 현재 파라미터를 바탕으로 각 데이터 포인트가 각 가우시안 분포에 속할 확률(소속 확률)을 계산
    - M 단계(Maximization): 소속 확률을 바탕으로 새로운 파라미터(평균, 공분산, 혼합 계수)를 추정
    - 파라미터의 변화가 일정 기준 이하로 떨어질 때까지 E 단계와 M 단계를 반복
    - 데이터 포인트를 가장 높은 소속 확률을 가진 가우시안 분포에 할당하여 클러스터링 완성

In [361]:
from sklearn.mixture import GaussianMixture

In [362]:
gmm = GaussianMixture(n_components=6)
gmm.fit(rfm_scaled)
gmm_clusters = gmm.predict(rfm_scaled)
rfm_df['gmm_clusters'] = gmm_clusters


In [363]:
# 실루엣점수 평가
silhouette_scores_gmm = metrics.silhouette_score(rfm_scaled, gmm_clusters)
print("GMM 실루엣 점수:", silhouette_scores_gmm)


GMM 실루엣 점수: 0.3754491532230954


In [364]:
# t-SNE 모델 생성 및 학습
tsne = TSNE(n_components=3, random_state=0)
rfm_tsne = tsne.fit_transform(rfm_scaled)

# plotly를 이용한 3D 시각화
fig = px.scatter_3d(x=rfm_tsne[:, 0], y=rfm_tsne[:, 1], z=rfm_tsne[:, 2], color=gmm_clusters, labels={'color': 'gmm_clusters'})
fig.update_layout(title='t-SNE 3D 시각화', scene=dict(xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Z')), width=800, height=800)
fig.show()

#### BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies)
- 특징
    - BIRCH 알고리즘은 대규모 데이터셋을 위해 설계된 클러스터링 알고리즘
    - 특히 메모리에 제약이 있는 상황에서 효율적인 데이터를 클러스터링하기 위해 계층적 방법을 사용하며, 클러스터링 과정에서 생성된 클러스터 트리(CF Tree)를 활용하여 데이터의 요약 정보 저장
    - BIRCH는 먼저 데이터를 소규모의 더 다루기 쉬운 부분집합으로 축소시키고, 이를 바탕으로 클러스터링을 수행합니다. 이 과정은 메모리 내에서 효율적으로 수행됨.
    - 주로 수치 데이터에 적합하며, 클러스터의 형태가 구형일 때 가장 잘 작동함. 다른 클러스터링 알고리즘과 결합하여 사용 가능.

- 주요 파라미터
    - 브랜치 요소(Branching Factor): CF Tree에서 노드가 가질 수 있는 최대 자식 수를 결정. 이 값은 메모리 사용량과 성능에 영향
    - 임계값(Threshold): 클러스터와 클러스터 간의 거리를 결정하는 데 사용되는 값으로, 이 값을 바탕으로 데이터 포인트가 현재 클러스터에 포함될지 여부를 결정
    - 노드당 데이터 포인트의 최대 수: 이 값은 CF Tree의 각 노드가 가질 수 있는 데이터 포인트의 최대 수를 결정.

- 수행전략
    - 데이터 포인트를 순차적으로 스캔하면서 CF Tree를 구축합니다. 이 트리는 클러스터에 대한 정보를 요약하여 저장.
    - CF Tree의 구축 과정에서 브랜치 요소와 임계값을 사용하여 데이터 포인트를 적절한 클러스터에 할당.
    - 트리가 구축되면, 계층적 클러스터링 방법을 사용하여 유사한 클러스터를 병합.
    - 필요한 경우, 최종 클러스터링 결과를 개선하기 위해 추가적인 클러스터링 알고리즘(K-Means 등)을 적용 하기도함.
- 단점
    - 매우 불균형한 데이터 분포나 클러스터의 크기 및 밀도 차이가 큰 경우에는 성능이 저하됨.

In [365]:
from sklearn.cluster import Birch

In [370]:
birch_clusters

array([0, 1, 2, 3, 4, 5])

In [371]:
birch = Birch(threshold=0.03, n_clusters=6)
birch.fit(rfm_scaled)
birch_result = birch.predict(rfm_scaled)
rfm_df['birch_clusters'] = birch_result


In [372]:
# 실루엣 점수 평가
silhouette_scores_birch = metrics.silhouette_score(rfm_scaled, rfm_df['birch_clusters'])
print("BIRCH 실루엣 점수:", silhouette_scores_birch)


BIRCH 실루엣 점수: 0.4247044216428314


#### OPTICS (Ordering Points To Identify the Clustering Structure)
- 특징
    - OPTICS 알고리즘은 데이터의 밀도 기반 클러스터링을 수행하며, DBSCAN의 확장, 개선한 형태.
    - 클러스터의 형태나 밀도가 균일하지 않은 데이터셋에 적합하며, 아웃라이어를 효과적으로 구분 가능.
    - OPTICS는 데이터 포인트 간의 밀도 연결성을 바탕으로 클러스터링 구조를 파악하며, 이를 통해 다양한 밀도를 가진 클러스터를 탐지.
    - 클러스터의 수를 미리 지정할 필요가 없으며, 클러스터링 결과는 '리치 가능성 플롯(Reachability Plot)'을 통해 시각화.

- 주요 파라미터
    - MinPts (최소 이웃 수): 핵심 포인트를 정의하는데 필요한 이웃의 최소 수
    - ε (엡실론): 주어진 포인트에서 이웃을 찾기 위한 거리의 최대 범위

- 수행전략
    - 각 데이터 포인트에 대해 '코어 거리(Core Distance)'와 '리치 가능성 거리(Reachability Distance)'를 계산.(이는 각 포인트가 클러스터의 일부로 간주될 수 있는지를 결정 기준)
    - 데이터 포인트를 리치 가능성 거리에 따라 정렬. (이 정렬 과정은 클러스터 구조를 파악하는 데 사용됨)
    - 리치 가능성 플롯을 생성하고 분석하여 클러스터의 경계 식별 플롯에서의 갑작스러운 값의 변화는 클러스터의 경계를 나타냄.

- 단점 : 계산 비용이 더 높음.

In [373]:
from sklearn.cluster import OPTICS

In [394]:
optics = OPTICS(eps=0.7, min_samples=5)
optics_result = optics.fit_predict(rfm_scaled)
rfm_df['optics_clusters'] = optics_result

In [395]:
# 실루엣 점수 평가
silhouette_scores_optics = metrics.silhouette_score(rfm_scaled, rfm_df['optics_clusters'])
print("OPTICS 실루엣 점수:", silhouette_scores_optics)


OPTICS 실루엣 점수: -0.3510622785425594


In [397]:
# Reachability Plot
reachability = optics.reachability_[optics.ordering_]

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(reachability)), y=reachability, mode='lines+markers', name='reachability'))
fig.update_layout(title='OPTICS Reachability Plot', xaxis_title='Index', yaxis_title='Reachability Distance')
fig.show()


#### Reachability Distance 해석 방법
- Reachability Distance의 낮은 값
    - 리치 가능성 거리가 낮은 값은 데이터 포인트들이 서로 가까이 위치해 있으며, 밀도가 높은 지역에 있다는 것을 나타냄. 이는 일반적으로 클러스터 내부를 의미.
- Reachability Distance의 급격한 상승
    - 플롯에서 리치 가능성 거리가 급격히 증가하는 지점은 클러스터 간의 경계를 나타내며, 서로 다른 클러스터 사이의 전환 지점. 이러한 급격한 변화는 클러스터들이 서로 잘 분리되어 있음을 의미.
- 평탄한 구간
    - 플롯에서 평탄하게 유지되는 구간은 동일한 클러스터 내의 데이터 포인트들을 나타냄. 이 구간에서 데이터 포인트들은 대체로 유사한 밀도를 가짐.
- 아웃라이어의 식별
    - 매우 높은 리치 가능성 거리를 가진 데이터 포인트는 다른 데이터 포인트들과 상당히 멀리 떨어져 있는 경우가 많으며, 이는 아웃라이어일 가능성이 높음.